In [ ]:
#hide
from asbe.core import *

# Automatic Stopping for Batch-mode Experimentation

> Code for using active learning and automatic stopping for designing experiments

Created with nbdev by ZP

## Install

`python -m pip install  git+https://github.com/puhazoli/asbe`

## How to use

In [ ]:
say_hello("Zoltan")

'Hello Zoltan!'